### TODO
- Hover verbessern (oben wird abgeschnitten, bessere eigenschaften anzeigen)
- Page title anpassen

In [351]:
import json, re, string

In [352]:
with open('data/dimlex.json') as j:
    dimlex_raw = json.load(j)

In [353]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [383]:
## Creates dictionary that saves necessary information about connector
conn_dict = {}
for entry in dimlex_raw['entry']:
    word_regex = '(^| )(' + entry['word']
    if '...' in word_regex:
        word_regex = re.sub('\s?\.{3}\s?',' [^.?!]+ ',word_regex)
    for orth in entry['orths']['orth']:
        orth_variant = ' '.join(part['t'] for part in orth['part'])
        word_regex += '|' + orth_variant
        # print(orth_variant)
    word_regex += ')( |\.|\?|!|$)'
    if word_regex not in conn_dict:
        # print(word_regex)
        conn_dict[word_regex] = []
        if entry['ambiguity']['non_conn']['t'] != 0:
            # print(entry['ambiguity']['non_conn'], entry['non_conn_reading'])
            if 'example' in entry['non_conn_reading']:
                if entry['non_conn_reading']['example'] != [[]]:
                    conn_dict[word_regex] = entry['non_conn_reading']['example']
            
    else: 
        print(dimlex_raw['entry'].index(entry), word_regex, entry['ambiguity']['non_conn'], 'already in dict')

In [386]:
html_string='<!DOCTYPE html>\n<html>\n<head>\n<title>ConnReco</title>\n</head>\n<body><div class="container">\n<p>\n<h1>' # start of paragraph
header_closed = False
with open("data/coronaimpfung.txt") as txt: # https://www.dw.com/de/coronaimpfung-die-m%C3%A4r-von-der-unfruchtbarkeit/a-58733733
    for line in txt:
        if line.strip() == '': 
            html_string+="</p>\n<p>"
            if header_closed == False:
                html_string += "</h1>"
                header_closed = True
            
        html_line = line
        for conn_reg in conn_dict:
            # print(conn_reg, conn_dict[conn_reg])
            ttt_list = []
            for entry in conn_dict[conn_reg]:
                ttt_list.append('{}: {}'.format(entry.get('type'), entry['t']))
            ttt = ' '.join(ttt_list)
            rr = '\g<2><span class="tooltiptext">{ttt}</span></span> '.format(ttt=ttt)
            if len(conn_dict[conn_reg]) == 0:
                rr = ' <span class="tooltip green">' + rr
            else: 
                rr = ' <span class="tooltip red">' + rr
            html_line = re.sub(conn_reg, r'{}'.format(rr), html_line)
        html_string+=html_line
        html_string+="\n"
html_string+="</div></body>\n</html>"

In [387]:
html = open('html/output.html', mode='w', encoding='utf8') 
html.write(html_string)
## add style: https://www.w3schools.com/css/css_tooltip.asp
html.write("""<style>\n
* {
  font-family: monospace;
}

.red {
  color: red;
}

.green {
  color: green;
}

.container {
  margin: 50px;
}

.tooltip {
  position: relative;
  display: inline-block;
  border-bottom: 1px dotted black;
}

.tooltip .tooltiptext {
  visibility: hidden;
  width: 120px;
  background-color: black;
  color: #fff;
  text-align: center;
  border-radius: 6px;
  padding: 5px 0;
  width: 120px;
  bottom: 100%;
  left: 50%;
  margin-left: -60px; /* Use half of the width (120/2 = 60), to center the tooltip */
  /* Position the tooltip */
  position: absolute;
  z-index: 1;
}

.tooltip:hover .tooltiptext {
  visibility: visible;
}

.tooltip .tooltiptext::after {
  content: " ";
  position: absolute;
  top: 100%; /* At the bottom of the tooltip */
  left: 50%;
  margin-left: -5px;
  border-width: 5px;
  border-style: solid;
  border-color: black transparent transparent transparent;
}

</style>""")
html.close()

In [278]:
html_string
conn_dict

'<!DOCTYPE html>\n<html>\n<head>\n<title>Page Title</title>\n</head>\n<body>\n<p>\nCoronaimpfung - die Mär von der Unfruchtbarkeit\n\n</p>\n<p>\n\nDas Gerücht ist nicht neu, hält sich <font color="red">aber</font> hartnäckig: Die Impfstoffe gegen COVID-19 können unfruchtbar machen – Männer <font color="red">wie</font> Frauen. Ist diese Sorge begründet? <font color="red">Und</font> <font color="red">wie</font> gefährlich ist <font color="red">dann</font> die COVID-Infektion selbst?\n\n"Die Impfung gegen das SARS-2 Virus macht unfruchtbar!" Solche <font color="red">und</font> ähnliche Aussagen  kursieren <font color="red">seit</font> Monaten in den sozialen Medien. \n\n</p>\n<p>\n\n <font color="red">Zunächst</font> hieß es, vor allem Frauen müssten um ihre Fortpflanzungsfähigkeit bangen – eine biologische Erklärung <font color="red">dafür</font> wurde <font color="red">auch</font> mitgeliefert: Die <font color="red">durch</font> den Impfstoff gebildeten Antikörper könnten nicht <font co

In [250]:
s = "hält sich aber hartnäckig aber."

In [251]:
re.sub('(^| )(aber|aber|Aber)( |\.|\?|!|$)', r' <font color=\"green\">\g<2></font> ', s)

'hält sich <font color=\\"green\\">aber</font> hartnäckig <font color=\\"green\\">aber</font> '

In [224]:
re.search('[\s^](aber|aber|Aber)[\\s\\.?!\\$]', s)